# 过拟合、欠拟合及其解决方案

## 训练误差和泛化误差

训练误差：模型在训练数据集上表现出来的误差。

泛化误差：指模型在测试数据上表现出的误差的期望，**并常常通过测试集上的误差近似（根据大数定律可知测试集样本数越大，近似效果越好）**，机器学习模型关注于降低泛化误差，这也是为什么要设置测试集。无法从训练误差估计泛化误差，所以一味的降低训练误差，并不意味着降低泛化误差。

## 模型选择

评估若干候选模型的表现，并从中选取模型。这一过程称为模型选择。

### 验证数据集

测试数据只能用于模型测试，不能用于调参。而且由于不能通过训练误差衡量泛化误差，所以不能仅依赖训练数据进行模型选择。鉴于此，我们可以预留一部分数据（验证数据集validation dataset)进行模型选择。例如，我们可以从训练集上选出一部分做验证数据，另外的作为真正的训练数据。

### k折交叉验证

把原始训练数据分为k个不重合的子集，然后我们做k次模型训练和验证。每一次用1个子集做验证集，另外k-1个子集做训练集。最后对k次的训练误差和验证误差分别做平均。

## 过拟合和欠拟合

- 模型无法得到较低的训练误差：欠拟合。

- 模型的训练误差远小于其测试误差：过拟合

可能导致这些问题的两个因素：模型复杂度和训练集大小。[VC维理论](https://tangshusen.me/2018/12/09/vc-dimension/)

### 模型复杂度

以多项式函数拟合为例，高阶多项式函数参数更多，选择空间更大，所以高阶多项式的模型复杂度也就更高。但同时也容易出现过拟合。

![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.11_capacity_vs_error.svg)

### 训练集大小

如果训练集中样本数过少（特别是少于参数个数时），过拟合更容易发生。（为什么？）同时泛化误差不会随着训练集增大而增大，所以，在计算资源允许的情况下，我们总是希望训练集大一些。


### 多项式函数拟合实验

`torch.cat`:torch.cat(tensors, dim=0, out=None) → Tensor:
>Concatenates the given sequence of seq tensors in the given dimension. All tensors must either have the same shape (except in the concatenating dimension) or be empty.

`torch.pow`:torch.pow(input, exponent, out=None) → Tensor:
>Takes the power of each element in input with exponent and returns a tensor with the result.

`torch.nn.Linear(in_features, out_features, bias=True)`:
>Applies a linear transformation to the incoming data: $y = xA^T + b$

`torch.utils.data.TensorDataset(*tensors)`:
>Dataset wrapping tensors.Each sample will be retrieved by indexing tensors along the first dimension.

`torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None)`:
>Data loader. Combines a dataset and a sampler, and provides an iterable over the given dataset.The DataLoader supports both map-style and iterable-style datasets with single- or multi-process loading, customizing loading order and optional automatic batching (collation) and memory pinning.


## 权重衰减

权重衰减等价于L2正则化。正则化通过给模型损失函数添加惩罚项使学出的模型参数值较小。L2范数惩罚项指的是模型权重参数每个元素的平方和与一个正的常数的乘积。该权重参数较大时，惩罚项在损失函数比重较大，学到的权重参数接近0。

MSE:
$$
 \ell(w_1, w_2, b) = \frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(x_1^{(i)} w_1 + x_2^{(i)} w_2 + b - y^{(i)}\right)^2 
$$

带有L2正则化的损失函数：

$$
\ell(w_1, w_2, b) + \frac{\lambda}{2n} |\boldsymbol{w}|^2,
$$

带有L2正则化时，权重的迭代方式：

$$
 \begin{aligned} w_1 &\leftarrow \left(1- \frac{\eta\lambda}{|\mathcal{B}|} \right)w_1 - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}}x_1^{(i)} \left(x_1^{(i)} w_1 + x_2^{(i)} w_2 + b - y^{(i)}\right),\\ w_2 &\leftarrow \left(1- \frac{\eta\lambda}{|\mathcal{B}|} \right)w_2 - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}}x_2^{(i)} \left(x_1^{(i)} w_1 + x_2^{(i)} w_2 + b - y^{(i)}\right). \end{aligned} 
$$


可见，$L_2$范数正则化令权重$w_1$和$w_2$先自乘小于1的数，再减去不含惩罚项的梯度。因此，$L_2$范数正则化又叫权重衰减。权重衰减通过惩罚绝对值较大的模型参数为需要学习的模型增加了限制，这可能对过拟合有效。

**如何实现**？
可以在构造优化器optimizer实例时通过`weight_dacay`参数指定权重衰减超参数。默认下，PyTorch会对权重和偏差同时衰减。我们可以分别对权重和偏差构造优化器实例，从而只对权重衰减。

```python
def fit_and_plot_pytorch(wd):
    # 对权重参数衰减。权重名称一般是以weight结尾
    net = nn.Linear(num_inputs, 1)
    nn.init.normal_(net.weight, mean=0, std=1)
    nn.init.normal_(net.bias, mean=0, std=1)
    optimizer_w = torch.optim.SGD(params=[net.weight], lr=lr, weight_decay=wd) # 对权重参数衰减
    optimizer_b = torch.optim.SGD(params=[net.bias], lr=lr)  # 不对偏差参数衰减

    train_ls, test_ls = [], []
    for _ in range(num_epochs):
        for X, y in train_iter:
            l = loss(net(X), y).mean()
            optimizer_w.zero_grad()
            optimizer_b.zero_grad()

            l.backward()

            # 对两个optimizer实例分别调用step函数，从而分别更新权重和偏差
            optimizer_w.step()
            optimizer_b.step()
        train_ls.append(loss(net(train_features), train_labels).mean().item())
        test_ls.append(loss(net(test_features), test_labels).mean().item())
    d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'loss',
                 range(1, num_epochs + 1), test_ls, ['train', 'test'])
    print('L2 norm of w:', net.weight.data.norm().item())
```

## 丢弃法(dropout)


丢弃法有不同的变体，本节提到的特指倒置丢弃法。

当对该隐藏层使用丢弃法时，该层的隐藏单元将有一定概率被丢弃掉。设丢弃概率为p，那么有p的概率$h_{i}$会被清零，有1−p的概率hi会除以1−p做拉伸。即丢弃法不改变其输入的期望值。

$h_{i}^{\prime}=\frac{\xi_{i}}{1-p} h_{i}$

$\because E\left(\xi_{i}\right)=1-p$

$\therefore E\left(h_{i}^{\prime}\right)=\frac{E\left(\xi_{i}\right)}{1-p} h_{i}=h_{i}$

![](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter03/3.13_dropout.svg)


即丢弃法不改变其输入的期望值。让我们对图3.3中的隐藏层使用丢弃法，一种可能的结果如图3.5所示，其中h2和h5被清零。这时输出值的计算不再依赖h5，在反向传播时，与这两个隐藏单元相关的权重的梯度均为0。由于在训练中隐藏层神经元的丢弃是随机的，即h1,…,h5都有可能被清零，输出层的计算无法过度依赖h1,…,h5中的任一个，从而在训练模型时起到正则化的作用，并可以用来应对过拟合。在测试模型时，我们为了拿到更加确定性的结果，一般不使用丢弃法。

### 实现dropout

```python
def dropout(X, drop_prob):
    X = X.float()
    assert 0 <= drop_prob <= 1
    keep_prob = 1 - drop_prob
    # 这种情况下把全部元素都丢弃
    if keep_prob == 0:
        return torch.zeros_like(X)
    mask = (torch.rand(X.shape) < keep_prob).float()

    return mask * X / keep_prob
```
- Assert语句
1、assert语句用来声明某个条件是真的。
2、如果你非常确信某个你使用的列表中至少有一个元素，而你想要检验这一点，并且在它非真的时候引发一个错误，那么assert语句是应用在这种情形下的理想语句。
3、当assert语句失败的时候，会引发一AssertionError。

- torch.rand(*size)
Returns a tensor filled with random numbers from a uniform distribution on the interval [0, 1).

`torch.matmul(input, other, out=None)`:Matrix product of two tensors.

### 简洁实现

在PyTorch中，我们只需要在全连接层后添加Dropout层并指定丢弃概率。在训练模型时，Dropout层将以指定的丢弃概率随机丢弃上一层的输出元素；在测试模型时（即model.eval()后），Dropout层并不发挥作用。

```python
net = nn.Sequential(
        d2l.FlattenLayer(),
        nn.Linear(num_inputs, num_hiddens1),
        nn.ReLU(),
        nn.Dropout(drop_prob1),
        nn.Linear(num_hiddens1, num_hiddens2), 
        nn.ReLU(),
        nn.Dropout(drop_prob2),
        nn.Linear(num_hiddens2, 10)
        )
```


# 梯度消失、梯度爆炸以及kaggle房价预测问题

## 梯度消失和梯度爆炸

深度模型有关数值稳定性的典型问题是消失（vanishing）和爆炸（explosion）。

**当神经网络的层数较多时，模型的数值稳定性容易变差。**

假设一个层数为$L$的多层感知机的第$l$层$\boldsymbol{H}^{(l)}$的权重参数为$\boldsymbol{W}^{(l)}$，输出层$\boldsymbol{H}^{(L)}$的权重参数为$\boldsymbol{W}^{(L)}$。为了便于讨论，不考虑偏差参数，且设所有隐藏层的激活函数为恒等映射（identity mapping）$\phi(x) = x$。给定输入$\boldsymbol{X}$，多层感知机的第$l$层的输出$\boldsymbol{H}^{(l)} = \boldsymbol{X} \boldsymbol{W}^{(1)} \boldsymbol{W}^{(2)} \ldots \boldsymbol{W}^{(l)}$。此时，如果层数$l$较大，$\boldsymbol{H}^{(l)}$的计算可能会出现衰减或爆炸。举个例子，假设输入和所有层的权重参数都是标量，如权重参数为0.2和5，多层感知机的第30层输出为输入$\boldsymbol{X}$分别与$0.2^{30} \approx 1 \times 10^{-21}$（消失）和$5^{30} \approx 9 \times 10^{20}$（爆炸）的乘积。当层数较多时，梯度的计算也容易出现消失或爆炸。

## 随机初始化模型参数

假设隐藏单元的激活函数相同，如果隐藏单元初始化参数相同，则隐藏单元的输出相同，梯度也相同，反向传播更新的参数也相同，这样相当于只有一个隐藏单元在发挥作用。所以我们要对模型参数，特别是权重参数进行随机初始化。

### pytorch中的随机初始化
`torch.nn.init.normal_()`使得模型`net`的权重参数以正态分布随机初始化。不过，PyTorch中`nn.Module`的模块参数都采取了较为合理的初始化策略（不同类型的layer具体采样的哪一种初始化方法的可参考[源代码](https://github.com/pytorch/pytorch/tree/master/torch/nn/modules)），因此一般不用我们考虑。

### Xavier初始化
假设dense layer的输入个数为a，输出个数为b，Xavier随机初始化将该层的权重参数随机采样于均匀分布：

$$
U\left(-\sqrt{\frac{6}{a+b}}, \sqrt{\frac{6}{a+b}}\right).
$$

它的设计主要考虑到，模型参数初始化后，每层输出的方差不该受该层输入个数影响，且每层梯度的方差也不该受该层输出个数影响。可参考这篇[blog](https://blog.csdn.net/shuzfan/article/details/51338178)